<font size="8"> **Adding environmental data from available observations to unique crabeater observations** </font>  
In this notebook, we will extract environmental data from sea ice and sea surface temperature observations using the coordinates of the reported crabeater seals sightings.

# Setting working directory
In order to ensure these notebooks work correctly, we will set the working directory. We assume that you have saved a copy of this repository in your home directory (represented by `~` in the code chunk below). If you have saved this repository elsewhere in your machine, you need to ensure you update this line with the correct filepath where you saved these notebooks.

In [1]:
import os
os.chdir(os.path.expanduser('~/Chapter2_Crabeaters/Scripts'))

# Loading other relevant libraries

In [2]:
from dask.distributed import Client
from glob import glob
#Accessing model data
import cosima_cookbook as cc
#Useful functions
import UsefulFunctions as uf
#Dealing with data
import xarray as xr
import pandas as pd
import numpy as np
#Data visualisation
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Paralellising work 

In [148]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 14,Total memory: 63.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42771,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 14
Started: Just now,Total memory: 63.00 GiB
Comm: tcp://127.0.0.1:40835,Total threads: 2
Dashboard: /proxy/43123/status,Memory: 9.00 GiB
Nanny: tcp://127.0.0.1:36603,


# Loading unique crabeater seal observations data frame

In [3]:
#Loading dataset as pandas data frame
crabeaters = pd.read_csv('../Biological_Data/unique_crabeater_obs_grid.csv')

#Ensuring date column is formatted correctly (year-month)
crabeaters['date'] = crabeaters.apply(lambda x: f'{x.year}-{str(x.month).zfill(2)}', axis = 1)

#Checking results
crabeaters

,event_date,date,latitude,longitude,source,year,sector,zone,yt_ocean,xt_ocean,month,season_year,life_stage,decade,presence
0,2019-03-09T00:00:00Z,2019-03,-64.9028,-62.8643,Belgica_121,2019,East Pacific,Antarctic,-64.888,-62.85,3,autumn,in-between,2010,1
1,2019-03-13T00:00:00Z,2019-03,-65.1413,-64.1443,Belgica_121,2019,East Pacific,Antarctic,-65.142,-64.15,3,autumn,in-between,2010,1
2,2019-03-15T00:00:00Z,2019-03,-65.1402,-64.0595,Belgica_121,2019,East Pacific,Antarctic,-65.142,-64.05,3,autumn,in-between,2010,1
3,2019-03-15T00:00:00Z,2019-03,-65.0977,-63.9802,Belgica_121,2019,East Pacific,Antarctic,-65.100,-63.95,3,autumn,in-between,2010,1
4,2019-03-14T00:00:00Z,2019-03,-65.3383,-64.1300,Belgica_121,2019,East Pacific,Antarctic,-65.353,-64.15,3,autumn,in-between,2010,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3235,2006-02-20T13:00:00Z,2006-02,-67.3064,73.1346,SCAR_seabirds,2006,Central Indian,Antarctic,-67.296,73.15,2,summer,moulting,2000,1
3236,2006-02-19T13:00:00Z,2006-02,-67.0479,69.9957,SCAR_seabirds,2006,Central Indian,Antarctic,-67.043,69.95,2,summer,moulting,2000,1
3237,2006-02-19T13:00:00Z,2006-02,-67.0665,69.9945,SCAR_seabirds,2006,Central Indian,Antarctic,-67.085,69.95,2,summer,moulting,2000,1
3238,2006-02-19T13:00:00Z,2006-02,-67.1395,69.9984,SCAR_seabirds,2006,Central Indian,Antarctic,-67.127,69.95,2,summer,moulting,2000,1


# Adding values for static variables only
Static variables referred to any physical variables that do not change over time (at least not during the time period of our interest). Examples include depth of the water column and distance to coastline. Given that we only have one value for these variables, the process of extracting data is relatively simple. We do not need to take into account the date observations were collected.

## Defining dictionary with information about static variables
This dictionary contains the column labels for each and the name of the files for each static variable to be included in our analysis. We will also define a variable containing the full path to the folder where all static variables are stored.

In [4]:
#Full path to static variables
base_dir_static = '/g/data/v45/la6889/Chapter2_Crabeaters/Static_Variables/'

#List of static variables
varDict = {'bottom_slope_deg': 'bathy_slope_GEBCO_2D.nc',
           'dist_shelf_km': 'distance_shelf.nc',
           'dist_coast_km': 'distance_coastline.nc',
           'depth_m': 'bathy_GEBCO_2D.nc'}

## Extracting data for each observation and adding it to a new column in crabeater data

In [5]:
#Getting coordinates from crabeater dataset
lat = xr.DataArray(crabeaters.latitude.values)
lon = xr.DataArray(crabeaters.longitude.values)

#Looping through dictionary keys
for var in varDict:
    #Creating full path to file of interest
    file_path = os.path.join(base_dir_static, varDict[var])
    #Load as raster
    ras = xr.open_dataarray(file_path)
    rename_var = ras.name
    #Extracting values
    ras_sub = ras.sel(xt_ocean = lon, yt_ocean = lat, method = 'nearest')
    #Turning into data frame and rounding all columns to 3 decimal places
    ras_df = ras_sub.to_dataframe().round(3).rename(columns = {rename_var: var})
    #Adding to crabeater observations data frame
    crabeaters = pd.merge(crabeaters, ras_df, on = ['yt_ocean', 'xt_ocean'], how = 'left', sort = True).drop_duplicates()
    
#Checking results
crabeaters

,event_date,date,latitude,longitude,source,year,sector,zone,yt_ocean,xt_ocean,month,season_year,life_stage,decade,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m
0,2004-02-19T00:00:00Z,2004-02,-77.697927,166.302939,GBIF_iNaturalist_Observations,2004,West Pacific,Antarctic,-77.687,166.35,2,summer,moulting,2000,1,89.980,606.014,32.155,176.033005
1,2014-01-13T14:51:22Z,2014-01,-76.859810,-36.055843,Filchner,2014,Atlantic,Antarctic,-76.842,-36.05,1,summer,moulting,2010,1,89.730,383.203,41.999,1064.319946
2,2000-01-06T00:00:00Z,2000-01,-76.775000,-157.290000,SCAR_Biology,2000,West Pacific,Antarctic,-76.758,-157.25,1,summer,moulting,2000,1,89.943,113.770,4.724,612.181030
3,2000-01-06T00:00:00Z,2000-01,-76.766000,-156.850000,SCAR_Biology,2000,West Pacific,Antarctic,-76.758,-156.85,1,summer,moulting,2000,1,89.898,113.279,2.642,656.099976
4,2020-02-02T00:00:00Z,2020-02,-76.362262,163.916020,GBIF_NA_Observations,2020,West Pacific,Antarctic,-76.378,163.95,2,summer,moulting,2020,1,89.871,603.311,21.027,519.267029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3897,1989-11-05T10:00:00Z,1989-11,-60.066700,70.200000,SCAR_seabirds,1989,Central Indian,Subantarctic,-60.047,70.25,11,autumn,weaning,1980,1,89.978,478.562,622.374,4235.085938
3898,2008-12-26T00:00:00Z,2008-12,-59.640000,9.115000,GBIF_IRSNB_Belgian Antarctic Research Expediti...,2008,Atlantic,Antarctic,-59.645,9.15,12,summer,weaning,2000,1,89.958,844.113,926.685,5462.943848
3899,1989-11-05T02:00:00Z,1989-11,-59.483300,71.866700,SCAR_seabirds,1989,Central Indian,Subantarctic,-59.493,71.85,11,autumn,weaning,1980,1,89.865,531.076,692.923,4498.345215
3900,2013-11-12T00:00:00Z,2013-11,-57.826121,-26.346061,GBIF_NA_Observations,2013,Atlantic,Antarctic,-57.829,-26.35,11,autumn,weaning,2010,1,NaN,1443.071,1067.077,506.523987


## Saving data frame with static variables
Given that the dynamic variables take some time to extract. We will save intermediary results to avoid having to extract them again.

In [6]:
#Defining output folder
folder_out = '../Environmental_Data/Env_obs'
#Checking folder exists
os.makedirs(folder_out, exist_ok = True)

#Saving file
crabeaters.to_csv(os.path.join(folder_out, 'unique_crabeater_obs_static_env.csv'), index = False)

# Adding values for dynamic variables
Given the amount of crabeater seal observations and the time period covered by this dataset, the extraction of these values may take some time. It is recommended to save the data frame after every time a new variable is extracted. This way we can avoid losing data.

In [7]:
crabeaters = pd.read_csv(os.path.join(folder_out, 'unique_crabeater_obs_static_env.csv'))
#Ensuring date column is formatted correctly (year-month)
crabeaters['date'] = crabeaters.apply(lambda x: f'{x.year}-{str(x.month).zfill(2)}', axis = 1)
crabeaters

,event_date,date,latitude,longitude,source,year,sector,zone,yt_ocean,xt_ocean,month,season_year,life_stage,decade,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m
0,2004-02-19T00:00:00Z,2004-02,-77.697927,166.302939,GBIF_iNaturalist_Observations,2004,West Pacific,Antarctic,-77.687,166.35,2,summer,moulting,2000,1,89.980,606.014,32.155,176.033
1,2014-01-13T14:51:22Z,2014-01,-76.859810,-36.055843,Filchner,2014,Atlantic,Antarctic,-76.842,-36.05,1,summer,moulting,2010,1,89.730,383.203,41.999,1064.320
2,2000-01-06T00:00:00Z,2000-01,-76.775000,-157.290000,SCAR_Biology,2000,West Pacific,Antarctic,-76.758,-157.25,1,summer,moulting,2000,1,89.943,113.770,4.724,612.181
3,2000-01-06T00:00:00Z,2000-01,-76.766000,-156.850000,SCAR_Biology,2000,West Pacific,Antarctic,-76.758,-156.85,1,summer,moulting,2000,1,89.898,113.279,2.642,656.100
4,2020-02-02T00:00:00Z,2020-02,-76.362262,163.916020,GBIF_NA_Observations,2020,West Pacific,Antarctic,-76.378,163.95,2,summer,moulting,2020,1,89.871,603.311,21.027,519.267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3235,1989-11-05T10:00:00Z,1989-11,-60.066700,70.200000,SCAR_seabirds,1989,Central Indian,Subantarctic,-60.047,70.25,11,autumn,weaning,1980,1,89.978,478.562,622.374,4235.086
3236,2008-12-26T00:00:00Z,2008-12,-59.640000,9.115000,GBIF_IRSNB_Belgian Antarctic Research Expediti...,2008,Atlantic,Antarctic,-59.645,9.15,12,summer,weaning,2000,1,89.958,844.113,926.685,5462.944
3237,1989-11-05T02:00:00Z,1989-11,-59.483300,71.866700,SCAR_seabirds,1989,Central Indian,Subantarctic,-59.493,71.85,11,autumn,weaning,1980,1,89.865,531.076,692.923,4498.345
3238,2013-11-12T00:00:00Z,2013-11,-57.826121,-26.346061,GBIF_NA_Observations,2013,Atlantic,Antarctic,-57.829,-26.35,11,autumn,weaning,2010,1,NaN,1443.071,1067.077,506.524


## Loading environmental data from observations

In [68]:
#Creating dictionary with useful information
varDict = {'var_original': 'dist_km',
           #Folder containing obs
           'obs_main': '/g/data/v45/la6889/Chapter2_Crabeaters/SeaIceObs/Distance_Edge/*.nc',
           #Name to store variable in final data frame
           'var_short': 'dist_ice_edge_km',
           #Output folder
           'base_out': '../Environmental_Data/Env_obs'}

In [69]:
#Getting list of all obs in folder
files_var = sorted(glob(varDict['obs_main']))
var = varDict['var_original']

#Loading all data into single dataset
var_df = xr.open_mfdataset(files_var)[var]
var_df.name = varDict['var_short']
if 'xt_ocean' not in var_df.coords:
    var_df = var_df.rename({'lon': 'xt_ocean', 'lat': 'yt_ocean'})

#Checking results
var_df

<xarray.DataArray 'dist_ice_edge_km' (time: 494, yt_ocean: 740, xt_ocean: 3600)>
dask.array<concatenate, shape=(494, 740, 3600), dtype=float64, chunksize=(1, 740, 3600), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) datetime64[ns] 1978-11-15 1978-12-15 ... 2019-12-15
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... -45.18 -45.11 -45.04
  * xt_ocean  (xt_ocean) float64 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
Attributes:
    units:      km
    long_name:  distance to nearest neighbour

## Subsetting variables by time to match observations
We will use the `latitude` and `longitude` columns together with the `date` column from the crabeater seal observations to find the corresponding grid cell in the model outputs and extract the value of the environmental factor of our interest.

In [70]:
#Getting coordinates from the crabeater data
lat = xr.DataArray(crabeaters.latitude)
lon = xr.DataArray(crabeaters.longitude)
#Getting data of observation from the crabeater data
time = xr.DataArray(crabeaters.apply(lambda x: pd.to_datetime(f'{x.date}-16'), axis = 1))

## Extracting data

In [71]:
#Extracting data
var_sub = var_df.sel(time = time, yt_ocean = lat, xt_ocean = lon, method = 'nearest')

#Transforming to data frame
var_pd = var_sub.to_dataframe().sort_values(['time', 'xt_ocean', 'yt_ocean'])
#Adding year and month
var_pd['year'] = var_pd.time.dt.year
var_pd['month'] = var_pd.time.dt.month
#Removing time column that is no longer needed
var_pd.drop(columns = 'time', inplace = True)
#Finding name of columns to round up
round_cols = [i for i in var_pd.columns if 'ocean' in i]
#Rounding coordinate values prior to merging
var_pd = var_pd.round({round_cols[0]: 3, round_cols[1]: 3})
#Getting column names for merging
cols = var_pd.drop(columns = varDict['var_short']).columns.tolist()

#Checking results
print(cols); var_pd

['yt_ocean', 'xt_ocean', 'year', 'month']


,yt_ocean,xt_ocean,dist_ice_edge_km,year,month
dim_0,,,,,
1467,-65.058,108.95,513.986550,1978,11
1514,-65.015,108.95,509.359268,1978,11
1189,-65.353,109.65,542.324041,1978,11
1255,-65.269,109.65,532.970089,1978,11
1704,-64.846,109.65,486.164848,1978,11
...,...,...,...,...,...
3241,-54.316,-36.45,637.971937,2019,12
4,-76.378,163.95,1163.734577,2019,12
251,-71.478,169.85,590.594893,2019,12


# Joining environmental data frame with crabeater data
We will use the grid cell coordinates and dates to perform this join.

In [72]:
crabeaters = crabeaters.merge(var_pd, on = cols, how = 'left')
crabeaters = crabeaters.drop_duplicates()
crabeaters

,event_date,date,latitude,longitude,source,year,sector,zone,yt_ocean,xt_ocean,...,decade,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m,SIC,SST_degC,lt_pack_ice,dist_ice_edge_km
0,2004-02-19T00:00:00Z,2004-02,-77.697927,166.302939,GBIF_iNaturalist_Observations,2004,West Pacific,Antarctic,-77.687,166.35,...,2000,1,89.980,606.014,32.155,176.033,NaN,-1.237460,0.000000,177.309322
1,2014-01-13T14:51:22Z,2014-01,-76.859810,-36.055843,Filchner,2014,Atlantic,Antarctic,-76.842,-36.05,...,2010,1,89.730,383.203,41.999,1064.320,0.885316,-1.623010,0.880952,712.582444
2,2000-01-06T00:00:00Z,2000-01,-76.775000,-157.290000,SCAR_Biology,2000,West Pacific,Antarctic,-76.758,-157.25,...,2000,1,89.943,113.770,4.724,612.181,0.010287,-0.901687,0.654762,862.799495
3,2000-01-06T00:00:00Z,2000-01,-76.766000,-156.850000,SCAR_Biology,2000,West Pacific,Antarctic,-76.758,-156.85,...,2000,1,89.898,113.279,2.642,656.100,0.010458,-0.911558,0.690476,866.721257
4,2020-02-02T00:00:00Z,2020-02,-76.362262,163.916020,GBIF_NA_Observations,2020,West Pacific,Antarctic,-76.378,163.95,...,2020,1,89.871,603.311,21.027,519.267,NaN,-1.403092,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3237,1989-11-05T10:00:00Z,1989-11,-60.066700,70.200000,SCAR_seabirds,1989,Central Indian,Subantarctic,-60.047,70.25,...,1980,1,89.978,478.562,622.374,4235.086,0.213187,-1.277349,0.035714,87.512018
3238,2008-12-26T00:00:00Z,2008-12,-59.640000,9.115000,GBIF_IRSNB_Belgian Antarctic Research Expediti...,2008,Atlantic,Antarctic,-59.645,9.15,...,2000,1,89.958,844.113,926.685,5462.944,0.418786,-1.270739,0.380952,294.557622
3239,1989-11-05T02:00:00Z,1989-11,-59.483300,71.866700,SCAR_seabirds,1989,Central Indian,Subantarctic,-59.493,71.85,...,1980,1,89.865,531.076,692.923,4498.345,0.155776,-1.218217,0.000000,33.037999
3240,2013-11-12T00:00:00Z,2013-11,-57.826121,-26.346061,GBIF_NA_Observations,2013,Atlantic,Antarctic,-57.829,-26.35,...,2010,1,NaN,1443.071,1067.077,506.524,0.155182,-1.245528,0.000000,11.858549


In [73]:
#Ensure output folder exists
os.makedirs(varDict['base_out'], exist_ok = True)

#Create file path where data will be saved
file_out = os.path.join(varDict['base_out'], 'unique_crabeater_obs_all_env.csv')
                        
crabeaters.to_csv(file_out, index = False)